In [ ]:
!pip install -q transformers==4.41.2
!pip install -q bitsandbytes==0.43.1
!pip install -q accelerate==0.31.0
!pip install -q langchain==0.2.5
!pip install -q langchainhub==0.1.20
!pip install -q langchain-chroma==0.1.1
!pip install -q langchain-community==0.2.5
!pip install -q langchain-openai==0.1.9
!pip install -q langchain_huggingface==0.0.3
!pip install -q chainlit==1.1.304
!pip install -q python-dotenv==1.0.1
!pip install -q pypdf==4.2.0
!npm install -g localtunnel
!pip install -q numpy==1.24.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.9/379.9 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 12.1 MB

In [37]:
code = """
import chainlit as cl
import torch
import os
from chainlit.types import AskFileResponse
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain_huggingface.llms import HuggingFacePipeline
from langchain.memory import ConversationBufferMemory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import ConversationalRetrievalChain
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=100)

embedding = HuggingFaceEmbeddings()
def process_file(file: AskFileResponse):
  if file.type == "text/plain":
    Loader = TextLoader
  elif file.type == "application/pdf":
    Loader = PyPDFLoader
  loader = Loader(file.path)
  documents = loader.load()
  docs = text_splitter.split_documents(documents)
  for i, doc in enumerate(docs):
    doc.metadata["source"] = f"source_{i}"
  return docs
def get_vector_db(file: AskFileResponse):
  docs = process_file(file)
  cl.user_session.set("docs", docs)
  vector_db = Chroma.from_documents(documents=docs, embedding=embedding)
  return vector_db

# ---- Use CPU ----
# def get_huggingface_llm(model_name: str = "lmsys/vicuna-7b-v1.5", max_new_tokens: int = 512):
#     # Load the model and tokenizer on CPU
#     model = AutoModelForCausalLM.from_pretrained(
#         model_name,
#         low_cpu_mem_usage=True,
#         device_map="cpu"  # Specify to use CPU
#     )

#     tokenizer = AutoTokenizer.from_pretrained(model_name)

#     model_pipeline = pipeline(
#         "text-generation",
#         model=model,
#         tokenizer=tokenizer,
#         max_new_tokens=max_new_tokens,
#         pad_token_id=tokenizer.eos_token_id,
#         device=-1  # Specify to use CPU
#     )

#     llm = HuggingFacePipeline(pipeline=model_pipeline)
#     return llm
# ---- Use GPU ----
def get_huggingface_llm(model_name: str = "lmsys/vicuna-7b-v1.5", max_new_tokens: int = 512):
    nf4_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=nf4_config,
        low_cpu_mem_usage=True
    )

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    model_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        device_map="auto"
    )

    llm = HuggingFacePipeline(pipeline=model_pipeline)
    return llm

LLM = get_huggingface_llm()

LLM = get_huggingface_llm()
welcome_message = '''Welcome to the PDF QA! To get started:
1. Upload a PDF or text file
2. Ask a question about the file
'''

@cl.on_chat_start
async def on_chat_start():
  files = None
  while files is None:
      files = await cl.AskFileMessage(
          content=welcome_message,
          accept=["text/plain", "application/pdf"],
          max_size_mb=20,
          timeout=180,
      ).send()
  file = files[0]

  msg = cl.Message(content=f"Processing '{file.name}'...",
                  disable_feedback=True)
  await msg.send()

  vector_db = await cl.make_async(get_vector_db)(file)

  message_history = ChatMessageHistory()
  memory = ConversationBufferMemory(
      memory_key="chat_history",
      output_key="answer",
      chat_memory=message_history,
      return_messages=True,)
  retriever = vector_db.as_retriever(search_type="mmr",
                                search_kwargs={"k": 3})

  chain = ConversationalRetrievalChain.from_llm(
      llm=LLM,
      chain_type="stuff",
      retriever=retriever,
      memory=memory,
      return_source_documents=True
  )

  msg.content = f"'{file.name}' processed. You can now ask questions!"
  await msg.update()

  cl.user_session.set("chain", chain)
@cl.on_message
async def on_message(message: cl.Message):
  chain = cl.user_session.get("chain")
  cb = cl.AsyncLangchainCallbackHandler()
  res = await chain.ainvoke(message.content, callbacks=[cb])
  answer = res["answer"]
  source_documents = res["source_documents"]
  text_elements = []

  if source_documents:
      for source_idx, source_doc in enumerate(source_documents):
          source_name = f"source_{source_idx}"
          text_elements.append(
              cl.Text(content=source_doc.page_content,
                      name=source_name)
          )
      source_names = [text_el.name for text_el in text_elements]

      if source_names:
          answer += f"\\nSources: {', '.join(source_names)}"
      else:
          answer += "\\nNo sources found"

  await cl.Message(content=answer, elements=text_elements).send()
"""
with open("app.py", "w") as file:
    file.write(code)

In [ ]:
!pip install pyngrok


In [ ]:
!ngrok authtoken *************************************************

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
!chainlit run app.py --host 0.0.0.0 --port 8000 &>/content/logs.txt


/bin/bash: line 1:  6014 Killed                  chainlit run app.py --host 0.0.0.0 --port 8000 &> /content/logs.txt


In [ ]:
!cat /content/logs.txt


2024-08-07 03:17:35.201691: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-07 03:17:35.237865: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-07 03:17:35.248590: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-07 03:17:35.276272: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-07 03:17:37.358848: W tensorflow/compiler/tf2

In [ ]:

import os
from pyngrok import ngrok

# Khởi động ngrok từ dòng lệnh
os.system("ngrok http 8000 &")

# Lấy URL công khai từ ngrok
url = ngrok.connect(8000)
print(f"Application running at {url}")

Application running at NgrokTunnel: "https://e79b-34-148-90-223.ngrok-free.app" -> "http://localhost:8000"
